In [8]:
import math
import numpy as np
import pandas as pd


In [9]:
def dataclean(x):
    for i in range(x.shape[1]-2):
        x.loc[:,str(i)] = x.loc[:,str(i)].str.replace('#','')
        x.loc[:,str(i)] = x.loc[:,str(i)].str.replace('x','')
        x.loc[:,str(i)] = x.loc[:,str(i)].str.replace('*','')
    x = x.replace({'NR':0})
    columns = []
    for i in range(x.shape[1]-2):
        columns.append(str(i))
    x[columns] = x[columns].astype(np.float)
    x = x.fillna(0)
    return x

In [10]:
def splitdata(x):
    y = x[x['測項'] == 'PM2.5'].iloc[:,2:]
    x = x.drop(['日期','測項'] , axis=1)
    y = y.to_numpy()
    x = x.to_numpy()
    newdata = x[0:18,:]
    for i in range(int(x.shape[0]/18-1)):
        newdata = np.hstack((newdata,x[18*(i+1):18*(i+2),:]))    
    y = y.reshape(-1,1)
    train_X = []
    for i in range(newdata.shape[1]-9):
        train_X.append(newdata[:,i:i+9])
    train_X = np.array(train_X)
    y = y[9:]
    return train_X,y

In [11]:
def valid(x, y):
    if y <= 2 or y > 100:
        return False
    if ((x <= 2).sum() + (x > 100).sum()) > 0:
        return False
    return True
def datafilter(x,y):
    xx = []
    yy = []
    for i in range(x.shape[0]):
        if valid(x[i][9],y[i]):
            xx.append(x[i].reshape(1,-1))
            yy.append(y[i])
    train_X = np.array(xx)
    train_Y = np.array(yy)
    train_Y = train_Y.reshape(-1,1)
    train_X = train_X.reshape(-1,162)        
    return train_X,train_Y       

In [21]:
def lse(train_X,train_Y):
    index = np.arange(train_X.shape[0])
    np.random.shuffle(index)
    train_X = train_X[index]
    train_Y = train_Y[index]
    import time
    w = np.full(162,0.1).reshape(-1,1)
    b = 0.1
    lr = 0.001
    lamda = 0.001
    itea = 10000
    sigma = np.zeros((162,1))
    sigmab = 0
    v = np.zeros((162,1))
    vb = 0
    beta1 = 0.9
    beta2 = 0.999
    eplision = 1e-8
    t = 0
    start = time.time()
    for i in range(itea):
        t += 1
        y_hat = np.dot(train_X,w) + b
        L = train_Y - y_hat
        w_grad = -2*np.dot(train_X.T,L) + 2*lamda*(w.sum())
        b_grad = 2*L.sum()
        sigma = beta2*sigma + (1-beta2)*w_grad*w_grad
        sigmab = beta2*sigmab + (1-beta2)*b_grad*b_grad
        v = beta1*v + (1-beta1)*w_grad
        vb = beta1*vb + (1-beta1)*b_grad
        sigman = sigma/(1-beta2**t)
        sigmabn = sigmab/(1-beta2**t)
        vn = v/(1-beta1**t)
        vbn = vb/(1-beta1**t)
    
        b = b - (lr*vbn)/(np.sqrt(sigmabn) + eplision)
        w = w - (lr*vn)/(np.sqrt(sigman)+ eplision)
    print(f'花了 {time.time()-start}秒') 
    return w,b

In [22]:
x1 = pd.read_csv('year1-data.csv')
x2 = pd.read_csv('year2-data.csv')
data = pd.concat([x1,x2])
data = dataclean(data)
x, y = splitdata(data)
x, y = datafilter(x,y)
w,b = lse(x,y)

花了 23.047420501708984秒


In [10]:
def rmse(x,y):
    mse = 0
    for i in range(len(y)):
        mse += np.sqrt((y[i]-x[i])**2)
        mse = mse/len(y)
    return mse

In [11]:
score = rmse(np.dot(x,w)+b,y)
print(score[0])

0.0004039347348357003


In [72]:
test = pd.read_csv('testing_data.csv')
test = dataclean(test)
test = test.drop(['id','測項'],axis=1)
test = test.to_numpy()
test = test.reshape(-1,162)

In [79]:
predict = pd.DataFrame(columns = ['id','value'])
ids = []
values = []
for i in range(500):
    ids.append('id_'+ str(i))
    values.append(float(np.dot(test,w)[i]+b))
predict['id'] = ids
predict['value'] = values  
predict.to_csv('predict.csv' , index = False)